In [1]:
import pymssql
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from datetime import date, timedelta
from prophet import Prophet
import datetime as dt

def DB_connection(tank_seq):
    conn = pymssql.connect(server='sql16ssd-014.localnet.kr', user='i2on11_admin', password='root0826', database='i2on11_admin')
    cursor = conn.cursor()

    conn_sql = 'SELECT signal_time, tank_remain_volume FROM gas_tank_volume_history where tank_seq = ' + tank_seq + 'ORDER BY signal_time DESC'

    # 쿼리 실행
    cursor.execute(conn_sql)

    # 결과 가져오기
    rows = cursor.fetchall()

    # 연결 닫기
    conn.close()
    days_ago_date = dt.date.today() - dt.timedelta(days=15)
    days_ago = dt.datetime.combine(days_ago_date, dt.datetime.min.time())

    d_h = pd.DataFrame(rows, columns=['ds', 'y'])
    d_h['ds'] = pd.to_datetime(d_h['ds'])
    d_h.set_index('ds', inplace=True)
    d_h.sort_index(ascending=True, inplace=True)
    d_h.drop(d_h[d_h['y'] <= 5].index, inplace=True)
    d_h.drop(d_h[d_h.index < days_ago].index, inplace=True)

    diff_d_h = d_h.copy()
    diff_d_h['y'] = diff_d_h['y'].diff()
    diff_d_h = diff_d_h.dropna()
    
    try:
        recent_datetime = diff_d_h.index[diff_d_h['y'] > 15]
        # print(recent_datetime)
        recent_datetime = recent_datetime.max()
        d_h.drop(d_h[d_h.index <= recent_datetime].index, inplace=True)
    except:
        pass
        
    d_h['ds']=d_h.index
    d_h['y']=d_h['y'].values
    
    diff_d_h['ds']=diff_d_h.index
    diff_d_h['y']=diff_d_h['y'].values
    return d_h, diff_d_h

In [2]:
def prophet_prediction(d_h, diff_d_h):
    # Prophet 객체 생성
    model = Prophet()

    # 데이터로 모델 훈련
    model.fit(d_h)

    # 향후 15일 동안의 예측
    future = model.make_future_dataframe(periods=15)
    forecast = model.predict(future)
#     fig1 = model.plot(forecast)
#     fig2 = model.plot_components(forecast)
#     print(forecast[['ds','yhat']])
    dh_index_max = d_h.index.max()
    latest_forecast = forecast[forecast['ds'] >= dh_index_max]
    prediction = {}
    for _, item in latest_forecast.iterrows():  # Iterate over rows using .iterrows()
        last_set = {item['ds']: item['yhat']}
        prediction.update(last_set)
    return prediction


In [3]:
def prediction_by_tank_seq(tank_seq):
    d_h, diff_d_h = DB_connection(tank_seq)
    pred_h = prophet_prediction(d_h, diff_d_h)
    return pred_h

prediction_by_tank_seq('327')


08:55:11 - cmdstanpy - INFO - Chain [1] start processing
08:55:13 - cmdstanpy - INFO - Chain [1] done processing


{Timestamp('2023-06-11 21:15:50'): 75.99998990297914,
 Timestamp('2023-06-12 21:15:50'): 75.99998456968696,
 Timestamp('2023-06-13 21:15:50'): 75.99997923639478,
 Timestamp('2023-06-14 21:15:50'): 75.9999739031026,
 Timestamp('2023-06-15 21:15:50'): 75.99996856981042,
 Timestamp('2023-06-16 21:15:50'): 75.99996323651824,
 Timestamp('2023-06-17 21:15:50'): 75.99995790322606,
 Timestamp('2023-06-18 21:15:50'): 75.99995256993387,
 Timestamp('2023-06-19 21:15:50'): 75.99994723664169,
 Timestamp('2023-06-20 21:15:50'): 75.99994190334951,
 Timestamp('2023-06-21 21:15:50'): 75.99993657005732,
 Timestamp('2023-06-22 21:15:50'): 75.99993123676514,
 Timestamp('2023-06-23 21:15:50'): 75.99992590347296,
 Timestamp('2023-06-24 21:15:50'): 75.99992057018078,
 Timestamp('2023-06-25 21:15:50'): 75.9999152368886,
 Timestamp('2023-06-26 21:15:50'): 75.99990990359642}

In [1]:
# import pandas as pd
# import datetime
# import pymssql

# conn1 = pymssql.connect(server='sql16ssd-014.localnet.kr', user='i2on11_admin', password='root0826', database='i2on11_admin')
# cursor1 = conn1.cursor()

# conn_sql = 'SELECT tank_seq FROM gas_tank'

# # 쿼리 실행
# cursor1.execute(conn_sql)

# # 결과 가져오기
# rows = cursor1.fetchall()

# # print(rows)
# exception = []
# excel_rows = []
# import datetime

# # Get the current date
# current_date = datetime.datetime.now().date()
# days_ago = current_date - datetime.timedelta(days=5)

# for tank_seq in rows:
#     tank_seq = str(tank_seq[0])
#     d_h = DB_connection(tank_seq)[0]
#     if len(d_h) > 3 and str(d_h.index.max()) >= str(days_ago):
#         print(tank_seq)
#         try:
#             pd_h = prediction_by_tank_seq(tank_seq)

#             for key, value in pd_h.items():
#                 row = [tank_seq, key, value]
#                 excel_rows.append(row)
#         except:
#             exception.append(tank_seq)
# conn1.close()



In [5]:
# col = set()
# for i in range(0, len(excel_rows)):
#     col.add(excel_rows[i][0])

In [6]:
# print(col)

{'750', '405', '327', '559', '374', '303', '367', '766', '558', '786', '401', '537', '368', '517', '313', '338', '375', '335', '527', '333', '510', '743', '396', '755', '380', '563', '353', '326', '356', '417', '304', '772', '372', '382', '383', '360', '523', '415', '532', '309', '526', '775', '754', '317', '400', '519', '336', '388', '554', '562', '518', '334', '325', '358', '376', '768', '320', '774', '528', '377', '406', '777', '761', '514', '312', '359', '515', '420', '571', '748', '783', '760', '392', '324', '397', '756', '395', '739', '378', '569', '477', '422', '513', '547', '379', '758', '354', '512', '757', '323', '503', '393', '339', '363', '373', '767', '794', '747', '348', '781', '366', '419', '364', '751', '759', '521', '381', '329', '542', '780', '310', '340', '398', '316', '370', '351', '347', '412', '773', '342', '403', '556', '744', '361', '391', '308', '414', '752', '568', '441', '561', '362', '525', '300', '541', '409', '529', '560', '394', '435', '337', '408', '357'

In [ ]:
from flask import Flask
from flask import request
from flask import jsonify
from datetime import datetime
from flask import render_template
import time
from multiprocessing.dummy import Pool as ThreadPool
import concurrent.futures

app = Flask(__name__)
executor = concurrent.futures.ThreadPoolExecutor()

@app.route('/test')
def seaqq():
    data = request.form
    start_time = time.time()

    tank_list = ['374', '338', '759', '347', '323', '437', '358', '396', '547', '503', '353', '512', '370', '327', '371']
    runningvalue = '<div> tank_list_length : ' + str(len(tank_list)) + '</div>'
    for tank_seq in tank_list:
        try:
            prophet = executor.submit(prediction_by_tank_seq(tank_seq))
            runningvalue = runningvalue + '<div>' + str(prophet) + '</div>'
        except:
            exception.append(str(tank_seq))
        
    end_time = time.time()
    exec_time = end_time - start_time
    
    return render_template('index.html', runningvalue=runningvalue, exec_time=exec_time, exception=exception)
#     return jsonify({'result': runningvalue})

@app.route('/test2')
def seaqq2():
    data = request.form
    exception = []
    start_time = time.time()

    tank_list = ['569', '317', '313', '316', '418', '419', '475', '743', '752', '324', '767', '380', '336', '513', '774']
    runningvalue = '<div> tank_list_length : ' + str(len(tank_list)) + '</div>'
    for tank_seq in tank_list:
        try:
            prophet = executor.submit(prediction_by_tank_seq(tank_seq))
            runningvalue = runningvalue + '<div>' + str(prophet) + '</div>'
        except:
            exception.append(str(tank_seq))
        
    end_time = time.time()
    exec_time = end_time - start_time
    
    return render_template('index.html', runningvalue=runningvalue, exec_time=exec_time, exception=exception)
#     return jsonify({'result': runningvalue})

@app.route('/test3')
def seaqq3():
    data = request.form
    exception = []
    start_time = time.time()
#     time.sleep(5)
    tank_list = ['395', '535', '510', '328', '405', '386', '315', '369', '392', '463', '477', '543', '394', '407', '341']
    runningvalue = '<div> tank_list_length : ' + str(len(tank_list)) + '</div>'
    for tank_seq in tank_list:
        try:
            prophet = executor.submit(prediction_by_tank_seq(tank_seq))
            runningvalue = runningvalue + '<div>' + str(prophet) + '</div>'
        except:
            exception.append(str(tank_seq))
        
    end_time = time.time()
    exec_time = end_time - start_time
    
    return render_template('index.html', runningvalue=runningvalue, exec_time=exec_time, exception=exception)
#     return jsonify({'result': runningvalue})
           
if __name__ == "__main__":
    app.run(host="0.0.0.0", port="5000")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.21:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jun/2023 09:23:29] "GET /test1 HTTP/1.1" 404 -
127.0.0.1 - - [12/Jun/2023 09:23:29] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Jun/2023 09:24:15] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [12/Jun/2023 09:24:15] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Jun/2023 09:24:38] "GET /test1 HTTP/1.1" 404 -
09:24:50 - cmdstanpy - INFO - Chain [1] start processing
09:24:50 - cmdstanpy - INFO - Chain [1] done processing
09:24:50 - cmdstanpy - INFO - Chain [1] start processing
09:24:50 - cmdstanpy - INFO - Chain [1] done processing
09:24:51 - cmdstanpy - INFO - Chain [1] start processing
09:24:51 - cmdstanpy - INFO - Chain [1] done processing
09:24:51 - cmdstanpy - INFO - Chain [1] start processing
09:24:51 - cmdstanpy - INFO - Chain [1] done processing
09:24:51 - cmdstanpy - INFO - Chain [1] start processing
09:24:51 - cmdstanpy - INFO - Chain [1] done processing
09:24:52 - 

09:31:08 - cmdstanpy - INFO - Chain [1] done processing
09:31:08 - cmdstanpy - INFO - Chain [1] done processing
09:31:08 - cmdstanpy - INFO - Chain [1] start processing
09:31:08 - cmdstanpy - INFO - Chain [1] done processing
09:31:08 - cmdstanpy - INFO - Chain [1] start processing
09:31:08 - cmdstanpy - INFO - Chain [1] start processing
09:31:08 - cmdstanpy - INFO - Chain [1] done processing
09:31:08 - cmdstanpy - INFO - Chain [1] done processing
09:31:08 - cmdstanpy - INFO - Chain [1] start processing
09:31:08 - cmdstanpy - INFO - Chain [1] done processing
09:31:08 - cmdstanpy - INFO - Chain [1] start processing
09:31:08 - cmdstanpy - INFO - Chain [1] start processing
09:31:08 - cmdstanpy - INFO - Chain [1] done processing
09:31:08 - cmdstanpy - INFO - Chain [1] done processing
09:31:08 - cmdstanpy - INFO - Chain [1] start processing
09:31:08 - cmdstanpy - INFO - Chain [1] done processing
09:31:09 - cmdstanpy - INFO - Chain [1] start processing
09:31:09 - cmdstanpy - INFO - Chain [1] 

09:32:36 - cmdstanpy - INFO - Chain [1] done processing
09:32:36 - cmdstanpy - INFO - Chain [1] start processing
09:32:36 - cmdstanpy - INFO - Chain [1] done processing
09:32:36 - cmdstanpy - INFO - Chain [1] start processing
09:32:37 - cmdstanpy - INFO - Chain [1] start processing
09:32:37 - cmdstanpy - INFO - Chain [1] done processing
09:32:37 - cmdstanpy - INFO - Chain [1] done processing
09:32:37 - cmdstanpy - INFO - Chain [1] start processing
09:32:37 - cmdstanpy - INFO - Chain [1] start processing
09:32:37 - cmdstanpy - INFO - Chain [1] start processing
09:32:37 - cmdstanpy - INFO - Chain [1] done processing
09:32:37 - cmdstanpy - INFO - Chain [1] done processing
09:32:37 - cmdstanpy - INFO - Chain [1] done processing
09:32:37 - cmdstanpy - INFO - Chain [1] start processing
09:32:37 - cmdstanpy - INFO - Chain [1] done processing
09:32:37 - cmdstanpy - INFO - Chain [1] start processing
09:32:37 - cmdstanpy - INFO - Chain [1] start processing
09:32:37 - cmdstanpy - INFO - Chain [1]